# Create a Pipeline that starts an inferencing service

In [ ]:
import json
from string import Template

import kfp
from kfp import dsl
from kfp.dsl.types import Float, String

In [ ]:
@dsl.pipeline(
    name="deploy_pipeline",
    description="""A pipeline that starts a serverless inferencing service.""",
)
def deploy(
    model_version: String = "",
    subdomain: String = ""
):
    jsontemplate = Template("""
    {
      "apiVersion": "serving.kubeflow.org/v1alpha2",
      "kind": "InferenceService",
      "metadata": {
        "name": "$subdomain",
        "namespace": "anonymous"
      },
      "spec": {
        "default": {
          "predictor": {
            "serviceAccountName": "sa",
            "pytorch": {
              "storageUri": "s3://mlpipeline/$model_version/",
              "modelClassName": "Net",
              "resources": {
                "limits": {
                  "cpu": "4.0",
                  "memory": "2Gi"
                },
                "requests": {
                  "cpu": "0.1",
                  "memory": "1Gi"
                }
              }
            }
          }
        }
      }
    }
    """)
    inference_service = jsontemplate.substitute({
      "model_version": model_version,
      "subdomain": subdomain
    })
    deployment = json.loads(inference_service)
    serve_op = dsl.ResourceOp(
        name="deploy",
        k8s_resource=deployment,
        success_condition="status.conditions.0.status == True"
    )

In [ ]:
kfp.compiler.Compiler().compile(deploy, "deployment.zip")

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<a href="/notebook/anonymous/my-notebook-server/files/kubeflow-aiops-workshop/notebooks/deployment.zip?download=1">Download</a>'))